import os
 
rootDir = '.'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    #for fname in fileList:
      #  print('\t%s' % fname)
    # Remove the first entry in the list of sub-directories
    # if there are any sub-directories present
    if len(subdirList) > 0:
        del subdirList[0]

In [1]:
import os
import pprint
import cmd
import sys
import datetime
import json
from webbrowser import open_new_tab
import itertools as it
import BTCInput2 as btc
#os.getcwd()

In [2]:
default_path = '/Users/thomassullivan/projects/GitHub/bulk_scotus/'

def wrapStringInHTMLMac3(program, url, body):
    import datetime
    from webbrowser import open_new_tab

    now = datetime.datetime.today().strftime("%Y%m%d-%H%M%S")
    filename = default_path+ program + '.html'
    f = open(filename,'w')

    wrapper = """<html>
    <head>
    <title>%s output - %s</title>
    </head>
    <body><p>URL: <a href=\"%s\">%s</a></p><p>%s</p></body>
    </html>"""

    whole = wrapper % (program, now, url, url, body)
    f.write(whole)
    f.close()

    #Change the filepath variable below to match the location of your directory
    filename = '/Users/thomassullivan/projects/GitHub/bulk_scotus/' + filename

    open_new_tab(filename)
    

In [19]:
'''Note: not all of these commands will end up being included in the final
command line interface.

Note for 2/7/20: Add WrappedHTMLFormat properties to each class that we have created.'''

class ScotusCMD(cmd.Cmd):
    '''
    The actual content of the commands should be moved to the app.py file.
    '''
    prompt='ScotusSearch'
    intro='Welcome to ScotusSearch'
    
    def do_hello(self, line):
        print('hello ', line)
        
    def do_build_directory(self, line):
        '''Build a directory for a given year'''
        #try:
        os.chdir(default_path)
        cent_values = {17: '/1700s', 18:'/1800s', 19:'/1900s', 20: '/2000s'}
        try:
            assert len(line) == 4, 'the year is four digits long'
            assert line.isnumeric() == True, 'must enter a year'
        except AssertionError:
            print('command entered incorrectly')
        try:
            century = int(line[:2]) #make it into an integer to get the key
        except ValueError:
            print('Incorrect year format')
        path = os.getcwd()
        path = path + cent_values.get(century)
        path2=path + '/'+line
        print('the path of the individual year is: {0}'.format(path2))
        cases = [i for i in os.listdir(path2)]
        print(cases)
        choice = btc.read_int_ranged('build directory (1-yes,2-no)? ',1,2)
        if choice == 1:
            os.chdir(path2)
            assert 'year_directory.json' not in os.listdir(path2)
            case_directory = {}
            for case in cases:
                try:
                    with open(case) as in_file:
                        new = json.load(in_file)
                        case_directory[new['id']] = new['citation']['case_name']
                except Exception as e:
                        print(e)
                        continue
            with open('year_directory.json', 'w') as out_file:
                json.dump(case_directory, out_file)
            os.chdir(default_path)
            print('directory built')
    
    def do_directory_search(self, line):
        os.chdir(default_path)
        cent_values = {17: '/1700s', 18:'/1800s', 19:'/1900s', 20: '/2000s'}
        try:
            assert len(line) == 4, 'the year is four digits long'
            assert line.isnumeric() == True, 'must enter a year'
        except AssertionError:
            print('command entered incorrectly')
        try:
            century = int(line[:2]) #make it into an integer to get the key
        except ValueError:
            print('Incorrect year format')
        path = os.getcwd()
        path = path + cent_values.get(century)
        path2=path + '/'+line
        print('the path of the individual year is: {0}'.format(path2))
        #cases = [i for i in os.listdir(path2)]
        #cases = [i for i in cases if i[-5] == '.json'] #load up the cases
        os.chdir(path2)
        with open('year_directory.json') as in_file:
            new_dir = json.load(in_file)
        #new_dir = json.load(path2+'year_directory.json')
        print(new_dir.keys())
        
        choice = btc.read_text('choose a case to view in html or "." to cancel')
        filename = str(new_dir[choice]) + '.json'
        if filename == '.':
            print('search cancelled')
            return
        #if filename in cases:
         #   os.chdir(path2)
          #  print(os.getcwd())

        with open(filename) as in_file:
            new_case = json.loads(in_file)
        print('new case:' ,new_case.keys())
        filename_choice = btc.read_text('enter filename to save or "." to cancel: ')
        if filename_choice == '.':
            print('file creation cancelled')
            os.chdir(default_path)
            return
        else:
            wrapStringInHTMLMac3(program=filename_choice, body=new_case['html_with_citations'], url='test')
            os.chdir(default_path)
            
            
    def do_search_year(self, line):
        #try:
        os.chdir(default_path)
        cent_values = {17: '/1700s', 18:'/1800s', 19:'/1900s', 20: '/2000s'}
        try:
            assert len(line) == 4, 'the year is four digits long'
            assert line.isnumeric() == True, 'must enter a year'
        except AssertionError:
            print('command entered incorrectly')
        try:
            century = int(line[:2]) #make it into an integer to get the key
        except ValueError:
            print('Incorrect year format')
        path = os.getcwd()
        path = path + cent_values.get(century)
        path2=path + '/'+line
        print('the path of the individual year is: {0}'.format(path2))
        cases = [i for i in os.listdir(path2)]
        cases = [i for i in cases if i[-5] == '.json']
        print(cases)
        choice = btc.read_text('choose a case to view in html or "." to cancel')
        filename = choice + '.json'
        if filename == '.':
            print('search cancelled')
            return
        if filename in cases:
            os.chdir(path2)
            print(os.getcwd())

            with open(filename) as in_file:
                new_case = json.loads(in_file)
            print('new case:' ,new_case.keys())
            filename_choice = btc.read_text('enter filename to save or "." to cancel: ')
            if filename_choice == '.':
                print('file creation cancelled')
                os.chdir(default_path)
                return
            else:
                wrapStringInHTMLMac3(program=filename_choice, body=new_case['html_with_citations'], url='test')
                os.chdir(default_path)
                
       # def do_directory_search():
            
    
    def do_EOF(self, line):
        return True
    
    def do_exit(self, arg):
        #del arg
        #a.close()
        print('Exiting Roundup Generator')
        sys.exit()

In [20]:
ScotusCMD().cmdloop()

Welcome to ScotusSearch


ScotusSearch directory_search 1803


the path of the individual year is: /Users/thomassullivan/projects/GitHub/bulk_scotus/1800s/1803


TypeError: the JSON object must be str, bytes or bytearray, not TextIOWrapper

In [ ]:
os.getcwd()

[Errno 21] Is a directory: '.ipynb_checkpoints'
directory built
1754
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1755'
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1756'
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1757'
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1758'
directory built
1759
directory built
1760
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1761'
directory built
1762
directory built
1763
directory built
1764
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1765'
directory built
1766
directory built
1767
directory built
1768
[Errno 2] No such file or directory: '/Users/thomassullivan/projects/GitHub/bulk_scotus/1700s/1769'
[Errno 2] No such file or directory: '/Users/thomassullivan/pro